In [30]:
import pandas as pd
import numpy as np
import random

In [31]:
df = pd.read_parquet("sales_cleaned.parquet", engine="pyarrow")

In [32]:
df

,price,condominium_fees,rooms,m2,bathrooms,disable_access,elevator,is_luxury,agency,floor,...,Impianto tv singolo,Armadio a muro,Impianto di allarme,Infissi esterni in doppio vetro / PVC,Giardino privato,Infissi esterni in doppio vetro / legno,Infissi esterni in vetro / legno,Caminetto,Giardino comune,Arredato
0,5500000.0,500.0,4,160.0,2,0.0,0.0,NaN,None,3.0,...,0,0,1,0,0,0,0,0,0,0
1,620000.0,150.0,3,80.0,2,0.0,0.0,NaN,None,1.0,...,0,0,0,1,0,0,0,0,0,0
2,420000.0,255.0,3,85.0,1,0.0,0.0,NaN,None,NaN,...,0,1,1,1,0,0,0,0,0,0
3,315000.0,90.0,2,72.0,2,0.0,0.0,NaN,None,NaN,...,0,1,0,1,0,0,0,0,0,0
4,5200000.0,1500.0,5+,385389.0,3+,0.0,0.0,NaN,None,4.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148243,295000.0,0.0,4,145.0,1,0.0,0.0,0.0,Affiliato Tecnorete: EPOREDIA UNO SAS,2.0,...,0,0,0,0,1,0,0,0,0,1
148244,99000.0,200.0,1,46.0,1,0.0,0.0,0.0,RE/MAX Brothers,0.0,...,1,0,0,0,0,0,1,0,1,1
148245,90000.0,200.0,1,46.0,1,0.0,0.0,0.0,RE/MAX Brothers,1.0,...,0,0,0,0,0,0,0,0,1,1
148246,79000.0,0.0,1,52.0,1,0.0,0.0,0.0,Affiliato Tecnocasa: STUDIO AOSTA SAS,0.0,...,0,0,0,0,0,0,0,0,1,1


In [33]:
df.columns

Index(['price', 'condominium_fees', 'rooms', 'm2', 'bathrooms',
       'disable_access', 'elevator', 'is_luxury', 'agency', 'floor', 'city',
       'district', 'street', 'contract', 'parking', 'heating', 'energy_class',
       'energy_consumption', 'timestamp', 'date_announcement', 'total_floors',
       'year_of_construction', 'condition', 'description', 'Porta blindata',
       'Balcone', 'Fibra ottica', 'Impianto tv centralizzato',
       'Esposizione esterna', 'Cancello elettrico', 'Esposizione doppia',
       'Terrazza', 'Cantina', 'VideoCitofono', 'Impianto tv singolo',
       'Armadio a muro', 'Impianto di allarme',
       'Infissi esterni in doppio vetro / PVC', 'Giardino privato',
       'Infissi esterni in doppio vetro / legno',
       'Infissi esterni in vetro / legno', 'Caminetto', 'Giardino comune',
       'Arredato'],
      dtype='object')

In [34]:
# Remove useless columns
cols = [
       'disable_access', 'agency', 'district', 'street',
       'district', 'street', 'contract', 'parking', 'heating', 'description',
       'energy_consumption', 'timestamp', 'date_announcement', 'total_floors',
       'Impianto tv centralizzato', 'Impianto tv singolo',
       'Armadio a muro', 'Infissi esterni in doppio vetro / PVC', 
       'Infissi esterni in doppio vetro / legno',
       'Infissi esterni in vetro / legno'
       ]

df = df.drop(columns=cols)

In [35]:
# Remove houses that are not in Milan
df = df[df["city"] == "Milano"]
df = df.drop(columns=["city"])
df.shape

(6920, 25)

In [36]:
# Remove houses that are too small or too big
df = df[df["m2"] <= 300]
df = df[df["m2"] >= 45]
df.shape

(6359, 25)

In [37]:
# Remove houses that are too cheap
df = df[df["price"] >= 75000]
df.shape

(6276, 25)

In [38]:
# Explore condominium fees of the houses for which
# I have the data, to compute an estimate for the relation m2-condominium_fees
# using the MEDIAN
ratio = [] # euro/m2

for index, row in df.iterrows():
    if not pd.isna(row["condominium_fees"]):
        condominium_fees = row["condominium_fees"]
        metrature_m2 = row["m2"]
        ratio.append( condominium_fees / metrature_m2 )

ratio_median = np.median(ratio)

df["condominium_fees"] = df["condominium_fees"].fillna(np.ceil(ratio_median * df["m2"]))

In [39]:
# Function to estimate the missing number of rooms from the metrature
# based on a very common mapping used by agencies (idealista, immobiliare.it, etc.)
def estimate_rooms(m2):
    m2_int = int(m2) # conversion string to int
    if m2 <= 45:
        return "1"
    elif 45 < m2 <= 70:
        return "2"
    elif 70 < m2 <= 100:
        return "3"
    elif 100 < m2 <= 140:
        return "4"
    elif 140 < m2 <= 180:
        return "5"
    else:
        return "5+"

In [40]:
# Function to estimate the number of bathrooms from metrature
def estimate_bathrooms(m2):
    m2_int = int(m2)
    if m2_int <= 70:
        return "1"
    elif 70 < m2_int <= 120:
        return "2"
    elif 120 < m2_int <= 180:
        return "3"
    else:
        return "3+"

In [41]:
# Fill None values for the "rooms" column
for index, row in df.iterrows():
    n_rooms = row["rooms"]
    m2 = row["m2"]
    if n_rooms == None:
        df.loc[index, "rooms"] = estimate_rooms(m2)

# Check there are no None left
d = df["rooms"].to_list()
unique = list(set(d))
print(unique)

['5', '3', '5+', '2', '4', '1']


In [42]:
# Fill 'None' values for the "bathrooms" column
for index, row in df.iterrows():
    row["bathrooms"]
    m2 = row["m2"]
    if row["bathrooms"] == 'None':
        df.loc[index, "bathrooms"] = estimate_bathrooms(m2)

# Check there are no None left
d = df["bathrooms"].to_list()
unique = list(set(d))
print(unique)

['3+', '2', '3', '1']


In [43]:
df

,price,condominium_fees,rooms,m2,bathrooms,elevator,is_luxury,floor,energy_class,year_of_construction,...,Cancello elettrico,Esposizione doppia,Terrazza,Cantina,VideoCitofono,Impianto di allarme,Giardino privato,Caminetto,Giardino comune,Arredato
30018,413000.0,168.0,2,72.0,1,0.0,NaN,1.0,A,None,...,1,0,0,0,1,0,0,0,0,0
30019,1062000.0,382.0,4,164.0,3,0.0,NaN,3.0,A,None,...,1,0,0,0,1,0,0,0,0,0
30020,696000.0,254.0,3,109.0,2,0.0,NaN,2.0,A,None,...,1,0,0,0,1,0,0,0,0,0
30021,502000.0,200.0,3,86.0,2,0.0,NaN,NaN,E,None,...,0,0,0,0,1,1,0,0,0,0
30022,610000.0,50.0,3,126.0,2,0.0,NaN,2.0,A,None,...,1,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147562,189000.0,150.0,2,60.0,1,1.0,0.0,2.0,F,1960,...,1,0,0,1,0,0,0,0,1,0
147568,680000.0,500.0,5,160.0,2,1.0,0.0,3.0,E,1970,...,1,1,0,1,0,0,0,0,1,0
147569,417000.0,60.0,3,70.0,1,0.0,0.0,0.0,D,1950,...,1,1,0,0,0,0,0,0,0,0
147571,1050000.0,625.0,5,163.0,2,1.0,1.0,6.0,E,1970,...,1,0,0,1,1,1,0,0,0,1


In [44]:
df.columns

Index(['price', 'condominium_fees', 'rooms', 'm2', 'bathrooms', 'elevator',
       'is_luxury', 'floor', 'energy_class', 'year_of_construction',
       'condition', 'Porta blindata', 'Balcone', 'Fibra ottica',
       'Esposizione esterna', 'Cancello elettrico', 'Esposizione doppia',
       'Terrazza', 'Cantina', 'VideoCitofono', 'Impianto di allarme',
       'Giardino privato', 'Caminetto', 'Giardino comune', 'Arredato'],
      dtype='object')

In [45]:
# Filling None values for year of construction
for index, row in df.iterrows():
    if row["year_of_construction"] == None:
        df.loc[index, "year_of_construction"] = random.randint(1960, 1980)

df_list = df["year_of_construction"].to_list()
unique = list(set(df_list))
print(unique)

['1912', '2017', '1950', '1600', '1942', '2023', '1996', '1903', '1955', '1910', '1988', '1959', '1994', '1935', '1911', '2000', '1938', '1887', '2007', '2016', '2001', '2021', '1925', '1937', '2018', '1967', '1961', '1947', '1929', '1860', '1890', '1930', '1500', '1974', '1995', '1800', '1907', '2011', '1984', '1980', '1936', '1927', '1908', '1971', '1953', '1977', '1915', '1900', '1972', '1992', '1966', '1991', '1100', '2027', '2020', '1400', '1928', '1954', '1990', '1940', '1904', '1970', '2003', '1964', '1962', '1946', '1997', '1952', '1931', '1926', '1979', '2022', '2006', '1985', '2013', '2005', '1969', '2004', '2009', '1998', '1960', '1948', '2014', '1976', '1913', '1987', '1958', '1978', '1956', '1934', '1939', '1853', '2019', '1920', '2012', '1973', '1963', '1945', '1700', '1993', '2024', '1975', '1957', '2026', '2015', 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, '1968', '2010', '1965', '1880', '

In [46]:
df["elevator"] = df["elevator"].fillna(0.0)

df["floor"] = df["floor"].fillna(2.0)

df["is_luxury"] = df["is_luxury"].fillna(0.0)

In [47]:
def estimate_energy_class(year):
    year = int(year)
    if year <= 1975:
        return 'G'
    elif year <= 1990:
        return 'F'
    elif year <= 2005:
        return 'E'
    elif year <= 2009:
        return 'D'
    elif year <= 2014:
        return 'C'
    elif year == 2015:
        return 'B'
    else:  # year >= 2016
        return 'A'

In [48]:
# Filling None values for energy class
for index, row in df.iterrows():
    if row["energy_class"] == None:
        df.loc[index, "energy_class"] = estimate_energy_class(row["year_of_construction"])

df_list = df["energy_class"].to_list()
unique = list(set(df_list))
print(unique)

['C', 'F', 'D', 'A', 'E', 'G', 'B']


In [49]:
for index, row in df.iterrows():
    if row["condition"] == None:
        df.loc[index, "condition"] = 'Buono / Abitabile'

df_list = df["condition"].to_list()
unique = list(set(df_list))
print(unique)

['Buono / Abitabile', 'Da ristrutturare', 'Ottimo / Ristrutturato', 'Nuovo / In costruzione', 'Partecipabile']


In [50]:
# Check if the final dataset contains any NaN, None or "None" (string) values
print(f"NaN: {df.isna().any().any()}")

print(f"None: {(df == None).any().any()}")

print(f'"None": {(df == None).any().any()}')

NaN: False
None: False
"None": False


In [51]:
df

,price,condominium_fees,rooms,m2,bathrooms,elevator,is_luxury,floor,energy_class,year_of_construction,...,Cancello elettrico,Esposizione doppia,Terrazza,Cantina,VideoCitofono,Impianto di allarme,Giardino privato,Caminetto,Giardino comune,Arredato
30018,413000.0,168.0,2,72.0,1,0.0,0.0,1.0,A,1964,...,1,0,0,0,1,0,0,0,0,0
30019,1062000.0,382.0,4,164.0,3,0.0,0.0,3.0,A,1962,...,1,0,0,0,1,0,0,0,0,0
30020,696000.0,254.0,3,109.0,2,0.0,0.0,2.0,A,1964,...,1,0,0,0,1,0,0,0,0,0
30021,502000.0,200.0,3,86.0,2,0.0,0.0,2.0,E,1978,...,0,0,0,0,1,1,0,0,0,0
30022,610000.0,50.0,3,126.0,2,0.0,0.0,2.0,A,1970,...,1,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147562,189000.0,150.0,2,60.0,1,1.0,0.0,2.0,F,1960,...,1,0,0,1,0,0,0,0,1,0
147568,680000.0,500.0,5,160.0,2,1.0,0.0,3.0,E,1970,...,1,1,0,1,0,0,0,0,1,0
147569,417000.0,60.0,3,70.0,1,0.0,0.0,0.0,D,1950,...,1,1,0,0,0,0,0,0,0,0
147571,1050000.0,625.0,5,163.0,2,1.0,1.0,6.0,E,1970,...,1,0,0,1,1,1,0,0,0,1


## Create mappings string -> code

In [52]:
df["rooms"] = df["rooms"].replace("5+", 6).astype(float)
df["bathrooms"] = df["rooms"].astype(float)

In [53]:
# Energy class
map = {"A": 1.0, "B": 2.0, "C": 3.0, "D": 4.0, "E": 5.0, "F": 6.0, "G": 7.0}

df["energy_class"] = df["energy_class"].replace(map)

# Year of construction
df["year_of_construction"] = df["year_of_construction"].astype(float)

/var/folders/xz/cl0qfkz56dl2q1b8vhqj2v180000gn/T/ipykernel_3605/829495106.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["energy_class"] = df["energy_class"].replace(map)


In [54]:
df.columns

Index(['price', 'condominium_fees', 'rooms', 'm2', 'bathrooms', 'elevator',
       'is_luxury', 'floor', 'energy_class', 'year_of_construction',
       'condition', 'Porta blindata', 'Balcone', 'Fibra ottica',
       'Esposizione esterna', 'Cancello elettrico', 'Esposizione doppia',
       'Terrazza', 'Cantina', 'VideoCitofono', 'Impianto di allarme',
       'Giardino privato', 'Caminetto', 'Giardino comune', 'Arredato'],
      dtype='object')

In [55]:
# Condition
map = {'Buono / Abitabile': float(0), 'Da ristrutturare': float(1), 'Nuovo / In costruzione': float(2), 'Partecipabile': float(3), 'Ottimo / Ristrutturato': float(4)}

df["condition"] = df["condition"].replace(map)

/var/folders/xz/cl0qfkz56dl2q1b8vhqj2v180000gn/T/ipykernel_3605/1096913396.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["condition"] = df["condition"].replace(map)


In [56]:
for index, row in df.iterrows():
    print(type(row["Porta blindata"]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.fl

In [57]:
print(df.dtypes)

price                   float64
condominium_fees        float64
rooms                   float64
m2                      float64
bathrooms               float64
elevator                float64
is_luxury               float64
floor                   float64
energy_class            float64
year_of_construction    float64
condition               float64
Porta blindata            int64
Balcone                   int64
Fibra ottica              int64
Esposizione esterna       int64
Cancello elettrico        int64
Esposizione doppia        int64
Terrazza                  int64
Cantina                   int64
VideoCitofono             int64
Impianto di allarme       int64
Giardino privato          int64
Caminetto                 int64
Giardino comune           int64
Arredato                  int64
dtype: object


In [58]:
# Save dataframe for later use
df.to_pickle("cleaned_df.pkl")